# Take raw i3 inputs and generate clean and formatted data
Kathleen Kennedy

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
## Import all data
# i3 raw data
ven_full_df = pd.read_csv('inputs/companies-export-18919.csv') # 2022 data
ipos_full_df = pd.read_csv('inputs/ipos-export-18925.csv') # 2022 data
ma_full_df = pd.read_csv('inputs/ma-export-18923.csv') # 2022 data
rel_full_df = pd.read_csv('inputs/relationships-export-18924.csv') # 2022 data

# data updates from Susanne Kurowski and Claudia Doblinger
ven_updates = pd.read_csv('inputs/updated_venture_data.csv')  # Binary failure and MA updates 
sus_df = pd.read_csv('inputs/foundingyr2.csv') # Year founded updates 

# data updates from author team
ven_founding_updates = pd.read_csv('inputs/companies-missing-year.csv') # Year founded updates 
ven_founding_updates = ven_founding_updates[['company','year.founded','year.founded.update']]
ven_fail_updates = pd.read_csv('inputs/venture_failure_updates.csv') # Year of failure updates 
ven_fail_updates = ven_fail_updates[['Company','Status_update','Failure Date_update',
                                     'Failure Date Source']].drop_duplicates()
inactive_updates = pd.read_csv('inputs/inactive_2-5yrs.csv') # Status updates
inactive_updates = inactive_updates[['Company','Status_update','Failure_date','IPO_date',
                                     'MA_date','Acquirer_update']]

ven_location_updates = pd.read_csv('inputs/author_location_updates.csv') # Location updates 
rel_updates = pd.read_csv('inputs/rel_date_update.csv') # Customer relationship updates
rel_updates = rel_updates[['Company','Relationship Partner','Relationship Date Update']]
rel_updates = rel_updates.dropna(subset=['Relationship Date Update'])

# companies to exclude based on relavancy filters in 0.preprocessing.R
excluded_df = pd.read_csv('inputs/companies-exclude-20220805.csv') # 2022 data
excluded = excluded_df['company'].to_list()

# cleaned investors data from 1.dataprocessing.R
cvc_df = pd.read_csv('inputs/investors_20220805.csv').drop_duplicates()
cvc_df = cvc_df.drop(['Unnamed: 0','i3.url','year.founded', 'industry.group', 'sector', 'tags',
                      'short.description', 'state', 'country', 'region',],axis=1)
cvc_df = cvc_df.fillna(value={'investment.year':-1})
cvc_df = cvc_df[cvc_df['investment.year'] < 2022]

# patent data
pat_df = pd.read_csv('inputs/patent_counts_2.csv') # output from matching_patents.R?

# ARPA-E data - prime awards and subawards from usaspending.gov, manually matched by authors
arpae_new = pd.read_csv('inputs/arpae_new.csv') 
arpae_upd = pd.read_csv('inputs/arpae_updates.csv')

# SBIR data - awards from SBIR API, output from sbir_award_search.R and sbir_i3_match.R
sbir_df = pd.read_csv('outputs/sbir_matches.csv')

#total_df = pd.read_csv('outputs/merged_venture_data.csv')

### Clean up venture dataset and add updates

In [3]:
ven_df = ven_full_df.drop(['i3 URL','Seeking Funding Date','Ticker Symbol','Record Created'],axis=1)
ven_df['company_lower'] = ven_df['Company'].apply(lambda x: x.lower())
ven_df = ven_df[~ven_df['company_lower'].isin(excluded)]

# Update year founded and filter (manual updates)
ven_df = ven_df.merge(ven_founding_updates,left_on='company_lower',right_on = 'company', how='left')
ven_df['Year Founded'] = ven_df.apply(lambda row: row['year.founded.update'] if pd.notnull(row['year.founded.update']) \
                                      else row['Year Founded'], axis = 1) 

ven_df = ven_df[ven_df['Year Founded'] >= 2005]
ven_df = ven_df[ven_df['Year Founded'] < 2021] # 2100 is default when no founding year could be found

# Update location and filter (manual updates from author team)
ven_df = ven_df.merge(ven_location_updates,left_on='company_lower',right_on='Company_to_merge',how='left')
ven_df["Location_source"] = ""

def update_location(row):
    if pd.notnull(row['Address_update']):
        address = row['Address_update']
    else: address = row['Address']
    if pd.notnull(row['City_update']):
        city = row['City_update']
    else: city = row['City']
    if pd.notnull(row['State_update']):
        state = row['State_update']
    else: state = row['State']
    if pd.notnull(row['Zip_update']):
        zipc = row['Zip_update']
    else: zipc = row['Zip']
    if pd.notnull(row['Country_update']):
        country = row['Country_update']
    else: country = row['Country']
    if pd.notnull(row['Region_update']):
        region = row['Region_update']
    else: region = row['Region']
    if pd.notnull(row['Location_source']):
        source = row['Location_source']
    else: source = ''
    return address, city, state, zipc, country, region, source 

ven_df.Address,ven_df.City,ven_df.State,ven_df.Zip,ven_df.Country,ven_df.Region,ven_df.Location_source = zip(
    *ven_df.apply(lambda row: update_location(row), axis=1))

ven_df = ven_df[ven_df['Country'] == 'United States']
ven_df = ven_df[~ven_df['State'].isin(('ON', 'QC', 'BC', 'MB'))] # remove Canadian entries mis-labeled as USA

# Update with previously cleaned data from Susanne Kurowski - failure date and zip code
sus_df = sus_df[['startup','yearfounded','s_time_death','s_death','zip']] 
sus_df['death date'] = sus_df['yearfounded'] + sus_df['s_time_death']
sus_df = sus_df[['startup','death date','zip']]
sus_df['startup'] = sus_df['startup'].apply(lambda x: x.lower())
ven_df = ven_df.merge(sus_df,left_on='company_lower',right_on = 'startup', how='left')

def update_zip(row):
    if ~np.isnan(row['zip']):
        return row['zip'],'Susanne data'
    return row['Zip'],row['Location_source']
ven_df['Zip'], ven_df['Location_source'] = zip(*ven_df.apply(lambda row: update_zip(row), axis=1))

## Update with manual data
# Status
ven_df = ven_df.merge(ven_fail_updates,on='Company',how='left')
ven_df['Status'] = ven_df.apply(lambda row: row['Status_update'] if pd.notna(row['Status_update']) \
                                      else row['Status'], axis = 1)

ven_df['Status'] = ven_df.Status.astype('str')

def check_fail(row):
    if row['Status'] == 'Out of Business':
        return 1
    if row['Status'] == 'Bankrupt':
        return 1
    if ~np.isnan(row['death date']):
        return 1
    return 0

# Failure dates and calculate age
#ven_df['Failure'] = ven_df.apply(lambda row: check_fail(row), axis=1)
ven_df['Failure Date'] = ven_df['death date']
ven_df['Failure Date'] = ven_df.apply(lambda row: row['Failure Date_update'] if pd.isnull(row['Failure Date']) \
                                      else row['Failure Date'], axis = 1)

ven_df = ven_df.drop(columns=['death date','zip','startup','Status_update','Failure Date_update',
                             'year.founded','year.founded.update','company','Company_to_merge',
                             'Address_update','City_update','State_update','Zip_update','Country_update',
                             'Region_update','Location_source','Failure Date Source'])
ven_df = ven_df.drop_duplicates().reset_index(drop=True)

ven_df['ID'] = ven_df.index + 1
id_dict = dict(zip(ven_df.company_lower, ven_df.ID))

### Clean up IPO, Merger/Acquisition, Corporate relationships, and Patents datasets

In [4]:
ipos_full_df = ipos_full_df[ipos_full_df['Year Founded'] >= 2005]
ipos_df = ipos_full_df.iloc[:,[0,15,16,17,18,19,20,21,22]]
ipos_df = ipos_df[ipos_df['IPO Status'] != 'Speculated']
ipos_df = ipos_df[ipos_df['IPO Status'] != 'Withdrawn']
ipos_df['IPO'] = 1
ipos_df = ipos_df.rename(columns = {'Date':'IPO Date','Amount Raised':'IPO Amount Raised'})
ipos_df = ipos_df[~ipos_df['IPO Date'].isna()] # Drop companies missing date after manual check
ipos_df['IPO Date'] = ipos_df['IPO Date'].apply(lambda x: str(x)[-4:]).astype(int)

In [5]:
ma_df = ma_full_df.drop_duplicates(subset=['Target','Acquirer','Announced'])
ma_df = ma_df.rename(columns={'Amount':'MA Amount','Notes':'MA Notes'})

# Fill in missing announced values with 0 
# verified manually that none of these startups are in our set of interest
values = {"Announced": '0'}
ma_df = ma_df.fillna(value=values)
ma_df['MA'] = 1

#ma_dups = ma_df[ma_df.duplicated(subset='Target',keep=False)].sort_values(by=['Target']).reset_index(drop=True)
#ma_dups.to_csv('outputs/duplicate_ma_entries.csv',index=False, encoding = "utf-8")

# Keep only the first MA entry for each startup
ma_df['Announced'] = ma_df['Announced'].apply(lambda x: str(x)[:4]).astype(int)
ma_df = ma_df.sort_values(by=['Target','Announced']).drop_duplicates(subset='Target',keep='first').reset_index(drop=True)

In [6]:
rel_df = rel_full_df.rename(columns={'Date':'Relationship Date','Source':'Relationship Source',
                                     'Notes':'Relationship Notes'})
rel_df = rel_df.drop(columns=['Country 1','Country 2','Sector 1','Sector 2'])

rel_a = rel_df[rel_df['Company 1'].isin(ven_df['Company'].tolist())]
rel_b = rel_df[rel_df['Company 2'].isin(ven_df['Company'].tolist())]

rel_a = rel_a.rename(columns = {'Company 1':'Company','Company 2': 'Relationship Partner'})
rel_b = rel_b.rename(columns = {'Company 1':'Relationship Partner','Company 2': 'Company'})

rel_to_merge = pd.concat([rel_a,rel_b]).drop_duplicates().reset_index(drop=True)

def update_rel(row):
    if ~np.isnan(row['Relationship Date Update']):
        return row['Relationship Date Update']
    return row['Relationship Date']

# Make a df that is just for relationship as an outcome - startup has a customer
rel_out = rel_to_merge.copy()
rel_out = rel_out[rel_out['Relationship Type']=='Customer']
rel_out.loc[:,'Customer Relationship'] = 1
rel_out = rel_out.drop_duplicates(subset='Company')
rel_out['company_lower'] = rel_out['Company'].apply(lambda x: x.lower())
rel_out = rel_out.merge(rel_updates,left_on = ['company_lower','Relationship Partner'], 
                        right_on = ['Company','Relationship Partner'], how='left')
rel_out['Relationship Date'] = rel_out.apply(lambda row: update_rel(row), axis=1)
values = {"Relationship Date": '2021/12/30'} 
rel_out = rel_out.fillna(value=values)
rel_out['Relationship Date'] = rel_out['Relationship Date'].apply(lambda x: str(x)[:4]).astype(int)
rel_out = rel_out.drop(columns=['Company_y','company_lower','Relationship Date Update'])
rel_out = rel_out.rename(columns={'Company_x':'Company'})

# Aggregate all relationship data to have a complete record
rel_to_merge = (rel_to_merge.groupby(by='Company')
                .agg({'Relationship Partner': lambda x: x.tolist(),
                      'Relationship Type': lambda x: x.tolist(),
                      'Relationship Date': lambda x: x.tolist()}).reset_index())
rel_to_merge.loc[:,'Relationship'] = 1

In [7]:
# Get total number of patents for each startup
pat_to_merge = (pat_df.groupby(by='Company').sum().
                drop(columns={'year'}).rename(columns={'pat_count':'patent_count'}))

### Combine investment data from i3 with additional government grant data from ARPA-E and SBIR

In [8]:
# Append new grants from ARPA-E and SBIR
arpae_new = arpae_new.rename(columns={'recipient_rename':'company'})
arpae_new['investor.firm'] = 'arpa-e'
cvc_df = (cvc_df.append(arpae_new).append(sbir_df)
          .drop(columns=['recipient_name','investment.amount.total']).drop_duplicates())
cvc_df['investment.amount'] = cvc_df['investment.amount'].astype(float)

arpae_upd = arpae_upd[['recipient.renamed','investor.firm.renamed',
                       'investment.year','investment.amount','i3.year']]
arpae_upd['investment.year'] = arpae_upd['investment.year'].astype(float)
arpae_upd['i3.year'] = arpae_upd['i3.year'].astype(float)
arpae_upd['investment.amount'] = arpae_upd['investment.amount'].astype(float)

# Set index for cvc_df to (company,investor.firm.renamed,investment.year)
cvc_df = cvc_df.set_index(['company','investor.firm.renamed','investment.year']).sort_index(level=0)

# Cycle through and update amounts
for i in np.arange(0,len(arpae_upd)):
    cvc_df.loc[(arpae_upd.loc[i]['recipient.renamed'],
                arpae_upd.loc[i]['investor.firm.renamed'],
                arpae_upd.loc[i]['i3.year']),'investment.amount']= arpae_upd.loc[i]['investment.amount']

# Set index for cvc_df to (company,investment.amount)
cvc_df = cvc_df.reset_index()
cvc_df = cvc_df.set_index(['company','investor.firm.renamed','investment.amount']).sort_index(level=0)

# Cycle through and update years
for i in np.arange(0,len(arpae_upd)):
    cvc_df.loc[(arpae_upd.loc[i]['recipient.renamed'],
               arpae_upd.loc[i]['investor.firm.renamed'],
               arpae_upd.loc[i]['investment.amount']),'investment.year'] = arpae_upd.loc[i]['investment.year']

cvc_df = cvc_df.reset_index()

C:\Users\kmkenne5\AppData\Local\Temp\ipykernel_13612\1674181182.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cvc_df = (cvc_df.append(arpae_new).append(sbir_df)
C:\Users\kmkenne5\AppData\Local\Temp\ipykernel_13612\1674181182.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cvc_df = (cvc_df.append(arpae_new).append(sbir_df)
C:\Users\kmkenne5\AppData\Local\Temp\ipykernel_13612\1674181182.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  cvc_df.loc[(arpae_upd.loc[i]['recipient.renamed'],


### Get investment data into format for regression

In [9]:
# Get year founded updates from venture dataframe
ven_merge = ven_df[['company_lower','Year Founded']].drop_duplicates()

# Version for logit analysis
inv_df = cvc_df.copy()

inv_df = inv_df.merge(ven_merge, how = 'left',left_on = 'company', right_on = 'company_lower')
inv_df = inv_df[inv_df['investment.year'] >= inv_df['Year Founded']]

inv_dict = {'financial sector':'other investors', 'private equity':'other investors', 
            'venture capital':'other investors','':'',
            'corporation or corporate venture':'corporation or corporate venture', 
            'public or quasi public':'public or quasi public',
            'angel investor':'other investors', 'family':'other investors', 
            'other':'other investors', 'accelerator or incubator':'other investors',
            'university':'other investors','crowdfunding':'other investors',
            'not an investor':'other investors','other investors':'other investors',np.nan:''}

inv_df['investor.type.edited'] = inv_df['investor.type.edited'].apply(lambda x: inv_dict[x])

def separate_grants(row):
    if row['investor.type.edited'] == 'public or quasi public':
        if row['investment.type'] == 'grant':
            return 'public or quasi public'
        else: 
            return 'other investors'
    else:
        return row['investor.type.edited']

inv_df['investor.type.edited'] = inv_df.apply(lambda row: separate_grants(row), axis = 1)

# For each investment, make binary variable for CVC, Other, Public Grants, and ARPA-E
def check_arpae(x):
    if x == 'arpa-e (old)':
        return 1
    if x == 'arpa-e':
        return 1
    if x =='advanced research projects agency - energy (arpa-e)':
        return 1
    else: 
        return 0

inv_df['ARPAE'] = inv_df['investor.firm'].apply(lambda x: check_arpae(x))

inv_df = pd.get_dummies(inv_df, columns = ['investor.type.edited'])
inv_df = inv_df.rename(columns = {'investor.type.edited_corporation or corporate venture':'CVC',
                                  'investor.type.edited_public or quasi public':'Public_grant',
                                  'investor.type.edited_other investors':'Other_inv'})

# Groupby and get counts of each investor type
inv_counts = inv_df.groupby('company').sum().reset_index()
inv_counts = inv_counts[['company','CVC','Other_inv','Public_grant','ARPAE']]
inv_counts = inv_counts.rename(columns = {'CVC':'CVC_count',
                                         'Public_grant':'Public_grant_count',
                                         'Other_inv':'Other_inv_count',
                                         'ARPAE':'ARPAE_count'})
# From counts, get binary variables
inv_counts['CVC_bin'] = inv_counts['CVC_count'].apply(lambda x: 1 if x > 0 else 0)
inv_counts['Public_grant_bin'] = inv_counts['Public_grant_count'].apply(lambda x: 1 if x > 0 else 0)
inv_counts['Other_inv_bin'] = inv_counts['Other_inv_count'].apply(lambda x: 1 if x > 0 else 0)
inv_counts['ARPAE_bin'] = inv_counts['ARPAE_count'].apply(lambda x: 1 if x > 0 else 0)
inv_df = inv_counts.copy()


# Combine all counts and binary varialbes to get a single line for each startup
#inv_df = inv_counts.merge(cvc_cat_counts,on='company',how='left')

# Version for recurrent event analysis
invs_df = cvc_df.copy()

invs_df = invs_df.merge(ven_merge, how = 'left',left_on = 'company', right_on = 'company_lower')
invs_df = invs_df[invs_df['investment.year'] >= invs_df['Year Founded']]

inv_dict = {'financial sector':'other investors', 'private equity':'other investors', 
            'venture capital':'other investors','':'',
            'corporation or corporate venture':'corporation or corporate venture', 
            'public or quasi public':'public or quasi public',
            'angel investor':'other investors', 'family':'other investors', 
            'other':'other investors', 'accelerator or incubator':'other investors',
            'university':'other investors','crowdfunding':'other investors',
            'not an investor':'other investors','other investors':'other investors',np.nan:''}
invs_df['investor.type.edited'] = invs_df['investor.type.edited'].apply(lambda x: inv_dict[x])

def separate_grants(row):
    if row['investor.type.edited'] == 'public or quasi public':
        if row['investment.type'] == 'grant':
            return 'public or quasi public'
        else: 
            return 'other investors'
    else:
        return row['investor.type.edited']
    
invs_df['investor.type.edited'] = invs_df.apply(lambda row: separate_grants(row), axis = 1)

invs_df = (pd.get_dummies(invs_df, columns=['investor.type.edited'])
            .groupby(['company','investment.type','investment.year'],as_index=False)
            .agg({'investor.type.edited_corporation or corporate venture':'sum',
                 "investor.type.edited_other investors":"sum",
                  "investor.type.edited_public or quasi public":"sum"}))
            
invs_df.rename(columns={'investor.type.edited_corporation or corporate venture':"CVC",
                        "investor.type.edited_other investors":"Other_investors",
                        "investor.type.edited_public or quasi public":"Pub_grant"}, inplace=True)
invs_df['CVC_bin'] = invs_df['CVC'].apply(lambda x: int(x != 0))
invs_df['Other_inv_bin'] = invs_df['Other_investors'].apply(lambda x: int(x != 0))
invs_df['Pub_grant_bin'] = invs_df['Pub_grant'].apply(lambda x: int(x != 0))


invs_df = invs_df.merge(ven_merge, left_on = 'company', right_on = 'company_lower', how='right')

invs_df['t_inv'] = invs_df['investment.year'] - invs_df['Year Founded']

invs_df = invs_df.drop(columns = ['company_lower','Year Founded'])

C:\Users\kmkenne5\AppData\Local\Temp\ipykernel_13612\1717285432.py:51: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  inv_counts = inv_df.groupby('company').sum().reset_index()


### Find last recorded activity for survival variable

In [10]:
# Find last investment in company
last_inv = (cvc_df.groupby('company').agg({'investment.year':'max'})
            .reset_index().rename(columns={'company':'Company'}))

# Last year a company received a patent
last_patent = pat_df.groupby('Company').agg({'year':'max'}).reset_index()

# Last year a company formed a new corporate relationship
rel_df = rel_full_df.rename(columns={'Date':'Relationship Date','Source':'Relationship Source',
                                     'Notes':'Relationship Notes'})
rel_df = rel_df.drop(columns=['Country 1','Country 2','Sector 1','Sector 2'])

rel_a = rel_df[rel_df['Company 1'].isin(ven_df['Company'].tolist())]
rel_b = rel_df[rel_df['Company 2'].isin(ven_df['Company'].tolist())]

rel_a = rel_a.rename(columns = {'Company 1':'Company','Company 2': 'Relationship Partner'})
rel_b = rel_b.rename(columns = {'Company 1':'Relationship Partner','Company 2': 'Company'})

last_rel = pd.concat([rel_a,rel_b]).drop_duplicates().reset_index(drop=True)
last_rel = last_rel.dropna(subset=['Relationship Date'])
last_rel['Relationship Date'] = last_rel['Relationship Date'].apply(lambda x: str(x)[:4]).astype(int)
first_rel = last_rel.groupby('Company').agg({'Relationship Date':'min'}).reset_index()
last_rel = last_rel.groupby('Company').agg({'Relationship Date':'max'}).reset_index()
last_rel['Company'] = last_rel['Company'].apply(lambda x: x.lower())

# Last year company experienced a success event
last_ipo = ipos_df.groupby('Company').agg({'IPO Date':'max'}).reset_index()
last_ipo['Company'] = last_ipo['Company'].apply(lambda x: x.lower())
last_ma = ma_df.groupby('Target').agg({'Announced':'max'}).reset_index()
last_ma['Company'] = last_ma['Target'].apply(lambda x: x.lower())


last_action = (last_ipo.merge(last_patent, on = 'Company', how = 'outer')
               .merge(last_rel, on='Company', how='outer')
               .merge(last_ma, on='Company', how='outer')
               .merge(last_inv, on='Company', how='outer'))
last_action = last_action.fillna(0)
last_action['last_act'] = last_action.apply(lambda row: max(row['IPO Date'],row['Relationship Date'],
                                                            row['year'],row['Announced'],
                                                            row['investment.year']),axis=1)
last_action = last_action.drop_duplicates()
last_action = last_action.drop(columns = ['IPO Date','year','Relationship Date',
                                          'Announced','Target','investment.year'])
last_action = last_action.rename(columns = {'Company':'company_lower'})
#last_action.to_csv('outputs/last_acts.csv')

### Merge all datasets for hazards analysis

In [86]:
hazard_df = ven_df.merge(ipos_df,on='Company',how='left')
hazard_df = hazard_df.merge(ma_df,left_on='Company',right_on='Target',how='left')
hazard_df = hazard_df.merge(rel_out,on='Company', how='left')
hazard_df['Company'] = hazard_df['Company'].apply(lambda x: x.lower())
hazard_df = hazard_df.merge(pat_to_merge,on='Company',how='left')
#inv_df for single line, invs_df for multi-line recurrent event analysis
hazard_df = hazard_df.merge(invs_df,left_on = 'Company', right_on = 'company',how='inner') 
hazard_df = hazard_df.merge(ven_updates, left_on='Company',right_on='orgname',how='left')
hazard_df = hazard_df.merge(last_action,on='company_lower',how='left')
hazard_df = hazard_df.merge(inactive_updates,on='Company',how='left')

# Fill in last action with founding year if blank
hazard_df['last_act'] = hazard_df.apply(lambda row: row['Year Founded'] if pd.isna(row['last_act']) 
                                        else row['last_act'], axis=1)

def update_status(row):
    if pd.notnull(row['Status_update']):
        status = row['Status_update']
    else: status = row['Status']
    if pd.notnull(row['Failure_date']):
        fail_date = int(row['Failure_date'])
    else: fail_date = row['Failure Date']
    if pd.notnull(row['IPO_date']):
        ipo_date = int(row['IPO_date'])
    else: ipo_date = row['IPO Date']
    if pd.notnull(row['MA_date']):
        ma_date = int(row['MA_date'])
    else: ma_date = row['Announced']
    if pd.notnull(row['Acquirer_update']):
        acquirer = row['Acquirer_update']
    else: acquirer = row['Acquirer']
    return status, fail_date, ipo_date, ma_date, acquirer

hazard_df.Status,hazard_df['Failure Date'],hazard_df['IPO Date'],hazard_df.Announced,hazard_df.Acquirer = zip(
    *hazard_df.apply(lambda row: update_status(row), axis=1))

# Update ma column with Susanne's data
def check_ma(row):
    if row['MA'] == 1:
        return 1
    if row['MA_update'] == 1:
        return 1
    if pd.notnull(row['Announced']):
        return 1
    return 0

hazard_df['MA'] = hazard_df.apply(lambda row: check_ma(row), axis=1)

def check_ipo(row):
    if row['IPO'] == 1:
        return 1
    if pd.notnull(row['IPO Date']):
        return 1
    return 0

hazard_df['IPO'] = hazard_df.apply(lambda row: check_ipo(row), axis=1)

def check_fail2(row):
    if row['Status'] == 'Out of Business':
        return 1
    if row['Status'] == 'Bankrupt':
        return 1
    if pd.notnull(row['Failure Date']):
        return 1
    return 0

hazard_df['Failure'] = hazard_df.apply(lambda row: check_fail2(row), axis = 1)

values = {'patent_count':0, 'IPO': 0, 'MA': 0, 'Relationship': 0,'sbti.status':0,'Failure':0,
          'Customer Relationship':0}
hazard_df = hazard_df.fillna(value=values)


def calc_success(row):
    if (row.IPO==1) and (row.MA==1):
        success = 1
        success_year = min(row['Announced'], row['IPO Date'])
        return success, success_year
    if (row.IPO==1):
        success = 1
        success_year = row['IPO Date']
        return success, success_year
    if (row.MA==1):
        success = 1
        success_year = row['Announced']
        return success, success_year
    success = 0
    success_year = ''  # Change to '' for multi-line recurrent event analysis
    return success, success_year
    
def calc_times(row,col):
    try:
        return row[col] - row['Year Founded']
    except:
        return ''
    
def calc_outcome(row):
    if row['Success'] == 1 and row['Failure'] == 1:
        return min(row['time_to_success'],row['time_to_failure'])
    if row['Success'] == 1:
        return row['time_to_success']
    if row['Failure'] == 1:
        return row['time_to_failure']
    else:
        return ''
        
# Make time_to_success, time_to_failure, time_to_customer variables
hazard_df['Success'], hazard_df['Success_year'] = zip(*hazard_df.apply(lambda row: calc_success(row), axis= 1))
hazard_df['time_to_success'] = hazard_df.apply(lambda row: calc_times(row,'Success_year'), axis = 1)
hazard_df['time_to_failure'] = hazard_df.apply(lambda row: calc_times(row,'Failure Date'), axis = 1)
hazard_df['time_to_rel'] = hazard_df.apply(lambda row: calc_times(row,'Relationship Date'), axis = 1)
hazard_df['time_to_ipo'] = hazard_df.apply(lambda row: calc_times(row,'IPO Date'), axis = 1)
hazard_df['time_to_ma'] = hazard_df.apply(lambda row: calc_times(row,'Announced'), axis = 1)
hazard_df['time_to_outcome'] = hazard_df.apply(lambda row: calc_outcome(row), axis = 1)

# Make location variable
# Make location binary variable = 1 if in Massachusettes, California, Colorado, DC as the hot spots from map
state_list = ['California','Colorado','Massachusetts','Washington DC']
hazard_df['Location'] = hazard_df['State'].apply(lambda x: 1 if x in state_list else 0)

def calc_age(row):
    if row['Failure'] == 1:
        return (int(row['Failure Date'])- int(row['Year Founded']) + 1)
    return (2021 - int(row['Year Founded']) + 1)

# Drop startups that only received investment in 2021
too_late = hazard_df[['ID','Company','investment.year']].groupby('Company').min()
too_late = too_late[too_late['investment.year'] > 2020]
too_late_list = too_late['ID'].tolist()
hazard_df = hazard_df[~hazard_df['ID'].isin(too_late_list)]

# Drop startups that only received investment before their founding date
too_early = hazard_df[['ID','Company','Year Founded','investment.year']].groupby('Company').max()
too_early = too_early[too_early['investment.year'] < too_early['Year Founded']]
too_early_list = too_early['ID'].tolist()
hazard_df = hazard_df[~hazard_df['ID'].isin(too_early_list)]

hazard_df['age'] = hazard_df.apply(lambda row: 2021 - row['Year Founded'], axis =1)

# Drop unneeded columns
hazard_df = hazard_df.drop(columns = ['Revenue Range','Revenue Range Source','Nbr Employees Range',
                                    'IPO Status','IPO Type','IPO Amount Raised','Shares Offered at IPO',
                                    'MA Amount','MA Notes','Relationship Source','Relationship Notes',
                                     'Status_update','Failure_date','IPO_date','MA_date','Acquirer_update',
                                     'MA_update','company_lower','orgname','Target','Failure_update'])

hazard_df.to_csv('outputs/hazard_data_recurrent.csv', index=False, encoding = 'utf-8')

### Merge all datasets together for plotting

In [88]:
total_df = ven_df.merge(ipos_df,on='Company',how='left')
total_df = total_df.merge(ma_df,left_on='Company',right_on='Target',how='left')
total_df = total_df.merge(rel_out,on='Company', how='left')
total_df['Company'] = total_df['Company'].apply(lambda x: x.lower())
total_df = total_df.merge(pat_to_merge,on='Company',how='left')
total_df = total_df.merge(ven_updates, left_on='company_lower',right_on='orgname',how='left')

total_df = total_df.drop(columns = ['MA_update','company_lower','orgname','Target','Failure_update'])
total_df = total_df.rename(columns={'country':'investor.country'})

values = {"IPO": 0, "MA": 0, "Relationship": 0,'sbti.status':0,'Failure':0}
total_df = total_df.fillna(value=values)

def separate_grants(row):
    if row['investor.type.edited'] == 'public or quasi public':
        if row['investment.type'] == 'grant':
            return 'public or quasi public'
        else: 
            return 'other investors'
    else:
        return row['investor.type.edited']

inv_dict = {'financial sector':'other investors', 'private equity':'other investors', 
            'venture capital':'other investors','':'',
            'corporation or corporate venture':'corporation or corporate venture', 
            'public or quasi public':'public or quasi public',
            'angel investor':'other investors', 'family':'other investors', 
            'other':'other investors', 'accelerator or incubator':'other investors',
            'university':'other investors','crowdfunding':'other investors',
            'not an investor':'other investors','other investors':'other investors',np.nan:''}
cvc_df['investor.type.edited'] = cvc_df['investor.type.edited'].apply(lambda x: inv_dict[x])

cvc_df['investor.type.edited'] = cvc_df.apply(lambda row: separate_grants(row), axis = 1)

total_df = total_df.merge(cvc_df,left_on = 'Company', right_on = 'company',how='left')

# Drop investments in 2021 to align with CPH methodology
total_df = total_df[total_df['investment.year'] < 2021]

# Drop investments before startup founding date
total_df = total_df[total_df['investment.year'] >= total_df['Year Founded']]

total_df.to_csv("outputs/merged_venture_data.csv", index=False, encoding = "utf-8")

### Summary numbers from final data

In [83]:
#print('Number of Investors: ' + str(len(cvc_df['investor.firm.renamed'].unique())))
print('Number of Companies: ' + str(len(hazard_df['Company'].unique())))
#print('Number of Deals: ' + str(len(inv_df[~inv_df['investor.firm.renamed'].isna()])))
print('Number of Patents: ' + str(pat_df['pat_count'].sum()))

Number of Companies: 3118
Number of Patents: 18214


### List of companies in final dataset

In [ ]:
tots = total_df.iloc[:,:14].drop_duplicates().reset_index(drop=True)
tots.to_csv("outputs/companies_from_merged_data.csv", index=False, encoding = "utf-8")